In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import yaml
import wandb 

#os.environ['WANDB_MODE'] = 'dryrun'

run_type = 'building-damage'
conf_file = 'config/config-damage-od.yaml'

wandb.init(project=run_type, config=yaml.load(open(conf_file)))
conf = wandb.config

In [ ]:
from glob import glob
from tqdm import tqdm_notebook as tqdm
import numpy as np
import torch
from torch import nn
from xv.util import vis_im_mask
from pprint import pprint

train_dir = '../../datasets/xview/train'
suppl_dir = '../../datasets/xview/tier3'
test_dir = '../../datasets/xview/test'

pprint(dict(conf))

In [ ]:
import albumentations as al
from albumentations import BboxParams

augment = al.Compose([
    al.HorizontalFlip(p=conf.aug_prob),
    al.VerticalFlip(p=conf.aug_prob),
    al.RandomRotate90(p=conf.aug_prob),
    al.Transpose(p=conf.aug_prob),
    al.RandomBrightnessContrast(p=conf.aug_prob),
    al.Rotate(p=conf.aug_prob),
],  bbox_params=BboxParams('pascal_voc', label_fields = ['labels'], min_visibility=conf.min_bbox_visibility))

In [ ]:
import random
import random
from xv import dataset
import pandas as pd

train_stems = pd.read_csv('config/train_stems.csv', header=None)[0]
dev_stems = pd.read_csv('config/dev_stems.csv', header=None)[0]

train_files = [f'{train_dir}/labels/{stem}_{conf.data_prefix}_disaster.json' for stem in train_stems]
dev_files = [f'{train_dir}/labels/{stem}_{conf.data_prefix}_disaster.json' for stem in dev_stems]

train_instances = dataset.get_instances(train_files, filter_none=conf.filter_none)
dev_instances = dataset.get_instances(dev_files, filter_none=conf.filter_none)

len(train_instances), len(dev_instances)

In [ ]:
if conf.add_suppl:
    train_instances *= conf.train_repeat
    suppl_files = glob(f'{suppl_dir}/labels/*{conf.data_prefix}_disaster.json')
    suppl_instances = dataset.get_instances(suppl_files, filter_none=conf.filter_none)
    train_instances += suppl_instances
    print(len(train_instances))

In [ ]:
train_dataset = dataset.DamageClassificationDataset(
    train_instances,
    conf.nclasses,
    augment=augment
)

dev_dataset = dataset.DamageClassificationDataset(
    dev_instances,
    conf.nclasses,
    augment=None
)

In [ ]:
def collate(batch):
    include = [len(bx) > 0 for _, bx, _ in batch]
    ims = torch.stack([torch.Tensor(ims) for ims, _, _ in batch])[include]
    bxs, clss = [], []
    for _, bx, cl in batch:
        if len(bx) == 0:
            continue
        bxs.append(torch.Tensor(bx))
        clss.append(torch.Tensor(cl))
    return ims, bxs, clss

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=conf.batch_size,
    shuffle=True,
    num_workers=10,
    collate_fn=collate
)

dev_loader = torch.utils.data.DataLoader(
    dev_dataset,
    batch_size=conf.batch_size,
    shuffle=False,
    num_workers=10,
    collate_fn=collate
)

In [ ]:
import random
import torch
from torch import nn
from xv.nn.nets import BoxClassifier

class MultiScaleResize(nn.Module):
    def __init__(self, scales = (0.5, 0.75, 1.)):
        super().__init__()
        self.scales = scales
    
    @torch.no_grad()
    def forward(self, x, boxes):
        scale = random.choice(self.scales)
        if scale != 1.:
            x = torch.nn.functional.interpolate(x, scale_factor=scale, mode='bilinear', align_corners=False)
            boxes = [b*scale for b in boxes]
        return x, boxes

In [ ]:
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone, BackboneWithFPN, resnet, FeaturePyramidNetwork
import segmentation_models_pytorch as smp

In [ ]:
# conf.backbone_norm = 'bn'

In [ ]:
# in_channels = smp.encoders.encoders[conf.backbone]['out_shapes']
# encoder = smp.encoders.get_encoder(conf.backbone, 'imagenet')
# backbone = nn.Sequential(encoder, FeaturePyramidNetwork(in_channels, 256))
# backbone.out_channels = 256
# , featmap_names=[4,3,2,1,0]

In [ ]:
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone

train_resize = MultiScaleResize(conf.training_scales)
backbone = resnet_fpn_backbone('resnet101', True)
model = BoxClassifier(backbone, conf.nclasses)
model = model.cuda()

In [ ]:
if conf.pretrain_weights:
    state_dict = torch.load(conf.pretrain_weights)
    model.load_state_dict(state_dict)

In [ ]:
optims = {'adam': torch.optim.Adam}
optim = optims[conf.optim](model.parameters(), lr=conf.lr)

In [ ]:
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optim, factor=conf.scheduler_factor, patience=conf.scheduler_patience
)

In [ ]:
from pytorch_toolbelt import losses

In [ ]:
from pytorch_toolbelt import losses
from torch.nn.modules.loss import CrossEntropyLoss
from xv.nn.losses import loss_dict, WeightedLoss
from torch.nn.modules.loss import CrossEntropyLoss


#loss_fn = WeightedLoss({loss_dict[l]():w for l, w in conf.loss_weights.items()})

#loss_fn = losses.JaccardLoss('multiclass')
#loss_fn = CrossEntropyLoss(weights)

if 'class_weight' in dict(conf):
    weights = torch.Tensor(conf.class_weight).float().cuda()
    loss_fn = CrossEntropyLoss(weights, reduction=conf.loss_reduce_mode)

In [ ]:
#loss_fn = losses.JointLoss(loss_fn, losses.FocalLoss(), 0.5, 0.5)

In [ ]:
from tqdm import tqdm_notebook as tqdm
import pdb
import logging

def run(model, optim, data, train_resize, loss_fn):
    model = model.train()
    total_loss = 0.
    for im, boxes, clss in tqdm(iter(data)):
        if im.shape[0] == 0:
            logging.warning("Empty batch.")
            continue
        im, boxes = train_resize(im, boxes)
        optim.zero_grad()
        out = model(im.cuda(), [b.cuda() for b in boxes])
        loss = loss_fn(out, torch.cat(clss).long().cuda())
        loss.backward()
        total_loss += loss
        optim.step()
    return {'train_loss': total_loss/len(train_loader)}

import scipy
from collections import defaultdict
from xv.run import get_metrics_for_counts

def weighted_tp_fp_fn(pred, targ, weights, c):
    tp = (np.logical_and(pred == c, targ == c) * weights).sum()
    fp = (np.logical_and(pred != c, targ == c) * weights).sum()
    fn = (np.logical_and(pred == c, targ != c) * weights).sum()
    return tp, fp, fn

@torch.no_grad()
def evaluate(model, data, nclasses, loss_fn):
    model.eval()
    nclasses = conf.nclasses
    loss_sum = 0.
    tps, fps, fns = [0. for _ in range(nclasses)], [0. for _ in range(nclasses)], [0. for _ in range(nclasses)]
    
    for im, boxes, clss in tqdm(data):
        res = im.shape[-1]
        out = model(im.cuda(), [b.cuda() for b in boxes])
        clss = torch.cat(clss).long()
        loss_sum += loss_fn(out, clss.cuda())

        out_ix = np.array(out.argmax(1).cpu())
        clss = clss.cpu().numpy()
        boxes_flat = torch.cat(boxes)
        areas = (boxes_flat[:,2] - boxes_flat[:,0]) * (boxes_flat[:,3] - boxes_flat[:,1])
        areas = areas.cpu().numpy()

        for cl in range(nclasses):
            tp, fp, fn = weighted_tp_fp_fn(out_ix, clss, areas, cl)
            tps[cl] += tp
            fps[cl] += fp
            fns[cl] += fn

    metrics = {}
    metrics['loss'] = loss_sum / len(data)
    
    aggregate = defaultdict(list)
    for ix in range(nclasses):
        categorical_ix_metrics =  get_metrics_for_counts(tps[ix], fps[ix], fns[ix])
        for k,v in categorical_ix_metrics.items():
            metrics[f'damage:categorical:{ix}:{k}'] = v
            aggregate[f'damage:categorical:{k}'].append(v)

    hmean = {f'hmean:{k}': scipy.stats.hmean(v) if all(v) else 0. for k,v in aggregate.items()}
    metrics.update(hmean)

    mean = {f'mean:{k}':scipy.mean(v) for k,v in aggregate.items()}
    metrics.update(mean)
    
    return metrics

In [ ]:
epoch, best_score = 0, 0

In [ ]:
for epoch in range(epoch, conf.epochs):
    metrics = {'epoch': epoch}
    train_metrics = run(model, optim, train_loader, train_resize, loss_fn)
    metrics.update(train_metrics)
    
    dev_metrics = evaluate(model, dev_loader, conf.nclasses, loss_fn)
    metrics.update(dev_metrics)
    
    wandb.log(metrics)
    scheduler.step(metrics['loss'])
    score = metrics[conf.metric]
    
    if score > best_score:
        torch.save(model.state_dict(), os.path.join(wandb.run.dir, "state_dict.pth"))
        best_score = score